In [1]:
# I did reference this page for some pointers here
# https://cypher.codes/writing/cryptopals-challenge-set-2

from random import choices, choice
import base64
from helpers import *
from Crypto.Cipher import AES

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]
        
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * ( npad % blocksize )

def EncryptECB(plaintext, key):
    BLOCKSIZE = 16
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    ECBcipher = AES.new(key, AES.MODE_ECB)
    blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cypher_blocks = [ ECBcipher.encrypt(block) for block in blocks ]
    return b''.join( ECBcipher.encrypt(block) for block in blocks )

def DecryptECB(ciphertext, key):
    ECBcipher = AES.new(key, AES.MODE_ECB)
    return ECBcipher.decrypt(ciphertext)

In [2]:
def MakeRandomKey(keysize = 16):
    return bytes( choices(range(0xFF), k = keysize) )

suffix = b'''
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
'''

print('Suffix size is ', len(base64.decodebytes(suffix)))

def EncryptionOracle(plaintext, verbose = False):
    key = b'B\xb7!\x9an\x92\xd6\xa2\xad\x0b\xa4\x97\x8d\xfeW\x0b'
    raw_suffix = base64.decodebytes(suffix)
    plaintext += raw_suffix
    print(f'Total input length: {len(plaintext)}.') if verbose else False


    ciphertext = EncryptECB(plaintext, key)
    return ciphertext

def DetectECB(ciphertext):
    BLOCKSIZE = 16
    assert len(ciphertext) % BLOCKSIZE == 0

    blocks = [ bytes(chunk) for chunk in Chunkerize(ciphertext, BLOCKSIZE, strict=False) ]
    unique_blocks = { *blocks }
    ratio = len(unique_blocks) / len(blocks)
    
    return ratio

Suffix size is  138


In [3]:
for n in range(64):
    plaintext = b'A' * n
    ciphertext = EncryptionOracle(plaintext)
    
    ratio = DetectECB(ciphertext)
    if ratio < 0.9999:
        print(f'Repeat block detected at n = {n}')
        break

Repeat block detected at n = 32


In [4]:
# Block repeats at 32, so blocksize = 32/2

In [5]:
BLOCKSIZE = 16

In [17]:
# We'll loop to get all the chars of the first block

unknown = []

for n in range(BLOCKSIZE):
    plaintext0 = b'A' * (BLOCKSIZE-n-1)
    ciphertext0 = EncryptionOracle(plaintext0)
    block0 = ciphertext0[:16]
    print(f'Block0 is {block0}')
    for char in range(256):
        plaintext = bytes([ *plaintext0, *unknown, char ])
        ciphertext = EncryptionOracle(plaintext)
        if ciphertext[:16] == block0:
            print(f'Found {char}')
            unknown += [char]
            break

Block0 is b'D\xce\xf0\x99\x02\x9e\x8fr8\xd2\xc7Yh-\x10,'
Found 82
Block0 is b'$\xd0o\xa3\x93:\xdb\xb4\xa2\xba\xb9\x8bg\xe7c1'
Found 111
Block0 is b'?\xb9\x95\x1d=\x03\xf85?M|fM\x97gA'
Found 108
Block0 is b'\xceou\x90D\xf9K\x94\x05N\xd2\xb2\xb9\xe4\x98C'
Found 108
Block0 is b'\xda\xa3\x00\xed.\xffv\xcb3N\xac\x17^\x80\xf8\x0c'
Found 105
Block0 is b'F\xe6\x85m\xbc\x8e6\xf9\xcf7eS\x92\x9b/\xa0'
Found 110
Block0 is b'\x08\x8e\xc8\xe2h\xd7\xd0\xd5{\xf8\xae\x91x\x02\xf2\xc1'
Found 39
Block0 is b'\xd9<"v\xf38\xa3\x87_\x03\x0c\xb0N\x80P\x97'
Found 32
Block0 is b'u\xe4\xa6k0\ru\x9b\x01X1`p;n\xc6'
Found 105
Block0 is b'P\xfd\x04\xce\xcex\xbc+\x0f\x07/\xbf\xa1\xae\n\x1f'
Found 110
Block0 is b'\x8d\xb6\x9f\x03\xd2\x1ap\xf7\xc9\xe3\xcfYQ\xcb\x8e\xa6'
Found 32
Block0 is b'%\xa41\x14\xf7\x9d\x92\x94k\xe4\x94\xf9\xf1N>\xdc'
Found 109
Block0 is b'\xb4\xf8\x17\xbd\x02^\xe2-\xdd\x12}\x12)\xa4#\xf3'
Found 121
Block0 is b'\t\x9b\x951\xb1z\xe5\xa4\xefq\xc1\xba\xe1\xd51Y'
Found 32
Block0 is b'\x19\xd8\x9a\x03

In [18]:
print(bytes(unknown))

b"Rollin' in my 5."


In [19]:
# Now let's do this for other blocks
# We'll need to begin by calculating the length of the unkown

unknown_length = len(EncryptionOracle(b''))
unknown_length
NBLOCKS = (unknown_length // BLOCKSIZE) + ( 1 if unknown_length % BLOCKSIZE  else 0 )
NBLOCKS

9

In [20]:
unknown = []
NBLOCKS = 9

for n in range(BLOCKSIZE*NBLOCKS):
    assert n == len(unknown)
    plaintext0 = b'A' * (NBLOCKS*BLOCKSIZE - 1 - n )
    ciphertext0 = EncryptionOracle(plaintext0)
    assert len(ciphertext0) % BLOCKSIZE == 0
    start = BLOCKSIZE * (NBLOCKS-1)
    end = BLOCKSIZE + start
    target_block = ciphertext0[start:end]
    assert len(target_block) == BLOCKSIZE
    for char in range(256):
        plaintext = bytes([ *plaintext0, *unknown, char ])
        assert len(plaintext) == BLOCKSIZE*NBLOCKS
        ciphertext = EncryptionOracle(plaintext)
        comparison_block = ciphertext[start:end]
        assert len(comparison_block) == BLOCKSIZE
        if comparison_block == target_block:
            print(f'Found {char}')
            unknown.append(char)
            break
        if char == 255:
            # If no match was found just fill with null
            unknown.append(0)
    

Found 82
Found 111
Found 108
Found 108
Found 105
Found 110
Found 39
Found 32
Found 105
Found 110
Found 32
Found 109
Found 121
Found 32
Found 53
Found 46
Found 48
Found 10
Found 87
Found 105
Found 116
Found 104
Found 32
Found 109
Found 121
Found 32
Found 114
Found 97
Found 103
Found 45
Found 116
Found 111
Found 112
Found 32
Found 100
Found 111
Found 119
Found 110
Found 32
Found 115
Found 111
Found 32
Found 109
Found 121
Found 32
Found 104
Found 97
Found 105
Found 114
Found 32
Found 99
Found 97
Found 110
Found 32
Found 98
Found 108
Found 111
Found 119
Found 10
Found 84
Found 104
Found 101
Found 32
Found 103
Found 105
Found 114
Found 108
Found 105
Found 101
Found 115
Found 32
Found 111
Found 110
Found 32
Found 115
Found 116
Found 97
Found 110
Found 100
Found 98
Found 121
Found 32
Found 119
Found 97
Found 118
Found 105
Found 110
Found 103
Found 32
Found 106
Found 117
Found 115
Found 116
Found 32
Found 116
Found 111
Found 32
Found 115
Found 97
Found 121
Found 32
Found 104
Found 105
Found 10

In [21]:
# Looks like we have the solution

bytes(unknown)

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01\x00\x00\x00\x00\x00"